## Exploratory Data Analysis ##
Although sensitivity seems an appropriate metric to us considering that false positives would not be detrimental to the city, our metric is accuracy based on the Kaggle competition. 

In [112]:
import pandas as pd
import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt

In [113]:
spray = pd.read_csv('./Molly-Work/all/spray.csv')
weather = pd.read_csv('./Molly-Work/all/weather.csv')
train = pd.read_csv('./Molly-Work/all/train.csv')
test = pd.read_csv('./Molly-Work/all/test.csv')

In [114]:
train.columns = [col.lower().replace(' ', '_') for col in train.columns]

In [115]:
test.columns = [col.lower().replace(' ', '_') for col in test.columns]

In [116]:
spray.columns = [col.lower().replace(' ', '_') for col in spray.columns]

In [117]:
weather.columns = [col.lower().replace(' ', '_') for col in weather.columns]

In [118]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
date         14835 non-null object
time         14251 non-null object
latitude     14835 non-null float64
longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [119]:
spray[spray.isnull().any(axis=1)]

,date,time,latitude,longitude
1030,2011-09-07,NaN,41.987092,-87.794286
1031,2011-09-07,NaN,41.987620,-87.794382
1032,2011-09-07,NaN,41.988004,-87.794574
1033,2011-09-07,NaN,41.988292,-87.795486
1034,2011-09-07,NaN,41.988100,-87.796014
1035,2011-09-07,NaN,41.986372,-87.794862
1036,2011-09-07,NaN,41.986228,-87.795582
1037,2011-09-07,NaN,41.984836,-87.793998
1038,2011-09-07,NaN,41.984836,-87.794670
1039,2011-09-07,NaN,41.984884,-87.795198


In [120]:
#Dropped time column from spray because it isn't contained in any other datasets.

In [121]:
spray.drop(columns='time', inplace=True)

In [122]:
spray.head()

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [123]:
spray.columns

Index(['date', 'latitude', 'longitude'], dtype='object')

In [124]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
station        2944 non-null int64
date           2944 non-null object
tmax           2944 non-null int64
tmin           2944 non-null int64
tavg           2944 non-null object
depart         2944 non-null object
dewpoint       2944 non-null int64
wetbulb        2944 non-null object
heat           2944 non-null object
cool           2944 non-null object
sunrise        2944 non-null object
sunset         2944 non-null object
codesum        2944 non-null object
depth          2944 non-null object
water1         2944 non-null object
snowfall       2944 non-null object
preciptotal    2944 non-null object
stnpressure    2944 non-null object
sealevel       2944 non-null object
resultspeed    2944 non-null float64
resultdir      2944 non-null int64
avgspeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [125]:
#Dropped all t-averages with missing value 'M' because there were additional missing values in each of these rows.

In [126]:
weather = (weather[weather['tavg'] != 'M'])

In [127]:
weather.shape

(2933, 22)

In [128]:
weather['tavg'] = weather['tavg'].astype(int)

In [129]:
weather[weather['depart'] == 'M']

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,HZ,M,M,M,0.00,29.46,30.12,12.9,6,13.2
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,,M,M,M,T,29.46,30.09,11.2,7,11.5
11,2,2007-05-06,68,52,60,M,30,46,5,0,...,,M,M,M,0.00,29.62,30.28,13.8,10,14.5
13,2,2007-05-07,84,50,67,M,39,53,0,2,...,,M,M,M,0.00,29.44,30.12,8.5,17,9.9
15,2,2007-05-08,80,60,70,M,57,63,0,5,...,HZ,M,M,M,T,29.36,30.02,2.5,8,5.4
17,2,2007-05-09,76,63,70,M,60,63,0,5,...,BR HZ,M,M,M,0.02,29.28,29.93,3.9,7,5.9
19,2,2007-05-10,83,59,71,M,52,61,0,6,...,BR HZ,M,M,M,0.00,29.26,29.91,2.0,9,3.9
21,2,2007-05-11,73,49,61,M,44,51,4,0,...,,M,M,M,0.00,29.39,30.03,11.7,36,12.8


In [130]:
weather['depart'].value_counts(normalize=True)

M      0.498125
 2     0.031708
-1     0.028640
-2     0.027276
 5     0.026253
 7     0.025912
 1     0.025912
 3     0.025571
 0     0.025230
-3     0.024548
 4     0.024207
 6     0.022844
 8     0.020116
-5     0.019434
-4     0.019093
-6     0.017047
 9     0.016025
10     0.015684
-8     0.014661
-7     0.010228
12     0.009547
11     0.009547
-9     0.008524
13     0.007842
14     0.007501
-10    0.007501
15     0.005114
16     0.004091
-11    0.003409
-12    0.002728
17     0.002387
-14    0.002046
18     0.002046
-13    0.001705
19     0.001364
20     0.001364
-16    0.001023
-15    0.001023
22     0.001023
21     0.000682
-17    0.000682
23     0.000341
Name: depart, dtype: float64

In [131]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2933 entries, 0 to 2943
Data columns (total 22 columns):
station        2933 non-null int64
date           2933 non-null object
tmax           2933 non-null int64
tmin           2933 non-null int64
tavg           2933 non-null int64
depart         2933 non-null object
dewpoint       2933 non-null int64
wetbulb        2933 non-null object
heat           2933 non-null object
cool           2933 non-null object
sunrise        2933 non-null object
sunset         2933 non-null object
codesum        2933 non-null object
depth          2933 non-null object
water1         2933 non-null object
snowfall       2933 non-null object
preciptotal    2933 non-null object
stnpressure    2933 non-null object
sealevel       2933 non-null object
resultspeed    2933 non-null float64
resultdir      2933 non-null int64
avgspeed       2933 non-null object
dtypes: float64(1), int64(6), object(15)
memory usage: 527.0+ KB


In [132]:
weather[weather.isnull().any(axis=1)]

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed


In [133]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
date                      10506 non-null object
address                   10506 non-null object
species                   10506 non-null object
block                     10506 non-null int64
street                    10506 non-null object
trap                      10506 non-null object
addressnumberandstreet    10506 non-null object
latitude                  10506 non-null float64
longitude                 10506 non-null float64
addressaccuracy           10506 non-null int64
nummosquitos              10506 non-null int64
wnvpresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.0+ KB


In [134]:
train[train.isnull().any(axis=1)]

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent


In [135]:
train.columns

Index(['date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'nummosquitos', 'wnvpresent'],
      dtype='object')

In [136]:
#Converted dates to datetime.

In [137]:
spray['date']   = pd.to_datetime(spray['date'])
weather['date'] = pd.to_datetime(weather['date'])
train['date']   = pd.to_datetime(train['date'])
test['date']    = pd.to_datetime(test['date'])

In [138]:
weather1 = weather[weather['station'] == 1]
weather2 = weather[weather['station'] == 2]
weather1.drop(columns='station', inplace=True)
weather2.drop(columns='station', inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [144]:
weather = weather1.merge(weather2, on='date')

In [145]:
train.merge(weather, on='date')

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,...,codesum_y,depth_y,water1_y,snowfall_y,preciptotal_y,stnpressure_y,sealevel_y,resultspeed_y,resultdir_y,avgspeed_y
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
5,2007-05-29,"1500 West Webster Avenue, Chicago, IL 60614, USA",CULEX RESTUANS,15,W WEBSTER AVE,T045,"1500 W WEBSTER AVE, Chicago, IL",41.921600,-87.666455,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
6,2007-05-29,"2500 West Grand Avenue, Chicago, IL 60654, USA",CULEX RESTUANS,25,W GRAND AVE,T046,"2500 W GRAND AVE, Chicago, IL",41.891118,-87.654491,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
7,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
8,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4
9,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,11,W CHICAGO,T049,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,...,BR HZ,M,M,M,0.00,29.44,30.09,5.8,16,7.4


In [146]:
spray.head()

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [23]:
train['trap'].head()

0    T002
1    T002
2    T007
3    T015
4    T015
Name: trap, dtype: object

In [24]:
train['addressnumberandstreet'].head()

0    4100  N OAK PARK AVE, Chicago, IL
1    4100  N OAK PARK AVE, Chicago, IL
2     6200  N MANDELL AVE, Chicago, IL
3      7900  W FOSTER AVE, Chicago, IL
4      7900  W FOSTER AVE, Chicago, IL
Name: addressnumberandstreet, dtype: object

In [25]:
train['address'].head()

0    4100 North Oak Park Avenue, Chicago, IL 60634,...
1    4100 North Oak Park Avenue, Chicago, IL 60634,...
2    6200 North Mandell Avenue, Chicago, IL 60646, USA
3      7900 West Foster Avenue, Chicago, IL 60656, USA
4      7900 West Foster Avenue, Chicago, IL 60656, USA
Name: address, dtype: object

In [27]:
train['species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: species, dtype: int64

In [28]:
train['wnvpresent'].head()

0    0
1    0
2    0
3    0
4    0
Name: wnvpresent, dtype: int64